In [21]:
import numpy as np 
import pandas as pd 
from ensemble_boxes import *

In [40]:
one=pd.read_csv('ens_the_ens_res_iou06.csv')
two=pd.read_csv('3826.csv')
three=pd.read_csv('3902.csv')        # 앙상블 할 csv불러오고, cand list에 추가.
cand=[one]
handin=pd.DataFrame()
data_len=len(one)
weights=[0.3797,0.3826,0.3902]

In [41]:
def str2bsl(plist):
    score=[]
    subscore=[]
    scores=[]
    sublabel=[]
    labels=[]
    label=[]
    box=[]
    subbox=[]
    boxes=[]
    for preds in plist:
        i=0
        raw=list(preds.strip(" ").split(" "))
        label=[]
        box=[]
        for nums in raw:
            if i%6==0:
                label=float(nums)
            elif i%6==1:
                score=float(nums)
            else:
                box.append(float(nums)/512)
                if i%6==5:
                    subbox.append(box)
                    sublabel.append(label)
                    subscore.append(score)
                    box=[]
                    score=0
                    label=0
            i+=1
        boxes.append(subbox)
        scores.append(subscore)
        labels.append(sublabel)
        subbox=[]
        subscore=[]
        sublabel=[]
    return boxes,scores,labels

In [42]:
SS=pd.DataFrame(index=range(len(one)))
SS['boxes']=[[] for i in range(data_len)]
SS['scores']=[[] for i in range(data_len)]
SS['labels']=[[] for i in range(data_len)]
for predict in cand:  
    box, score,label= str2bsl(predict['PredictionString'])
    for i in range(len(box)):
        SS.loc[i,'boxes'].append(box[i])
        SS.loc[i,'scores'].append(score[i])
        SS.loc[i,'labels'].append(label[i])

In [46]:
SS

,boxes,scores,labels
0,"[[[0.3148036301171875, 0.1217656955078125, 0.5...","[[0.09558909, 0.02947535, 0.01655788, 0.098732...","[[1.0, 1.0, 1.0, 2.0, 8.0, 10.0]]"
1,"[[[0.6104782819726563, 0.12442132830078124, 0....","[[0.03786198, 0.03766004, 0.02391933, 0.018942...","[[1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 4.0, 5.0,..."
2,"[[[0.1307811737109375, 0.3131836652734375, 0.7...","[[0.15082636, 0.04453334, 0.01373459, 0.051671...","[[1.0, 1.0, 2.0, 6.0, 6.0, 8.0, 8.0, 10.0]]"
3,"[[[0.5879351496679688, 0.30883440375, 0.697765...","[[0.14800087, 0.2212189, 0.21516512, 0.0197822...","[[2.0, 8.0, 8.0, 8.0, 8.0]]"
4,"[[[0.7515379190429687, 0.5228844881054687, 0.9...","[[0.18359709, 0.06896895, 0.04515188, 0.035230...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
...,...,...,...
832,"[[[0.11515939236328125, 0.28097063302734376, 0...","[[0.109358, 0.05482599, 0.05259762, 0.03644742...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0,..."
833,"[[[0.74099725484375, 0.7165795564648437, 0.860...","[[0.15459174, 0.01849138, 0.01733647, 0.017148...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0,..."
834,"[[[0.00099469162109375, 0.14015671611328126, 0...","[[0.15443978, 0.09123194, 0.08239905, 0.076745...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
835,"[[[0.2563033998046875, 0.692589581015625, 0.29...","[[0.10548609, 0.0402571, 0.02531943, 0.0214058...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."


In [49]:
prediction_strings=[]
for k in range(data_len):
    boxes_list=SS.loc[k,'boxes']
    scores_list=SS.loc[k,'scores']
    labels_list=SS.loc[k,'labels']
    iou_thr=0.6
    sigma = 0.1
    boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=None, iou_thr=iou_thr)
    boxes=boxes*512
    res=[]
    final=[]
    for i in range(len(scores)):
        res.append(labels[i])
        res.append(scores[i])
        res=res + list(boxes[i])
        final.append(res)
        res=[]
    f = np.array(sorted(final, key = lambda x : (x[0], -x[1])))
    f=f.flatten()
    f=f.tolist()
    prediction_strings.append(''.join(str(f)).strip('],[').replace(',',''))

Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Che

In [33]:
prediction_strings=[]
for k in range(data_len):
    boxes_list=SS.loc[k,'boxes']
    scores_list=SS.loc[k,'scores']
    labels_list=SS.loc[k,'labels']
    iou_thr=0.6
    skip_box_thr = 0.0001
    sigma = 0.1
    boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
    boxes=boxes*512
    res=[]
    final=[]
    for i in range(len(scores)):
        res.append(labels[i])
        res.append(scores[i])
        res=res + list(boxes[i])
        final.append(res)
        res=[]
    f = np.array(sorted(final, key = lambda x : (x[0], -x[1])))
    f=f.flatten()
    f=f.tolist()
    prediction_strings.append(''.join(str(f)).strip('],[').replace(',',''))

125, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
C:\Users\black\miniconda3\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 1.0, 0.074090578125, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
C:\Users\black\miniconda3\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.013592888671875, 1.0, 0.096473603515625, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
C:\Users\black\miniconda3\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 1.0, 0.05906064453125, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
C:\Users\black\miniconda3\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0019640509765625, 1.0, 0.081484056640625, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
C:\Users\black\minico

In [50]:
handin['PredictionString']=prediction_strings
handin['image_id']=one['image_id']

In [51]:
prediction_strings=[]
res=[]
for things in handin['PredictionString']:
    i=0
    raw=list(things.strip(" ").split(" "))
    sack=[]
    for nums in raw:
        if i%6==0:
            label=str(nums[0:-2])
        elif i%6==1:
            score="{:.8f}".format(float(nums))
        else:
            sack.append("{:.8f}".format(float(nums)))
            if i%6==5:
                box=' '.join(sack)
                sack=[]
                prediction_strings.append(label+' '+score+' '+box)
        i+=1
    res.append(' '.join(prediction_strings))
    prediction_strings=[]
handin['PredictionString']=res

In [52]:
handin

,PredictionString,image_id
0,1 0.09558909 161.17945862 62.34403610 257.2328...,batch_01_vt/0021.jpg
1,1 0.03786198 312.56488037 63.70372009 350.5556...,batch_01_vt/0028.jpg
2,1 0.15082636 66.95996094 160.35003662 375.3268...,batch_01_vt/0031.jpg
3,2 0.14800087 301.02279663 158.12321472 357.255...,batch_01_vt/0032.jpg
4,1 0.18359709 384.78741455 267.71685791 480.000...,batch_01_vt/0070.jpg
...,...,...
832,1 0.10935800 58.96160889 143.85696411 424.9673...,batch_03/0947.jpg
833,1 0.15459174 379.39059448 366.88873291 440.613...,batch_03/0968.jpg
834,1 0.15443978 0.50928211 71.76023865 72.6541061...,batch_03/0969.jpg
835,1 0.10548609 131.22734070 354.60586548 152.595...,batch_03/0992.jpg


In [53]:
handin.to_csv('ens_the_ens_res_iou06_self_nms.csv', index=None)